In [1]:
!pip install torch torchvision torchaudio
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
!pip install opencv-python-headless

Cloning into 'yolov5'...
remote: Enumerating objects: 16512, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 16512 (delta 41), reused 49 (delta 15), pack-reused 16408
Receiving objects: 100% (16512/16512), 15.17 MiB | 29.82 MiB/s, done.
Resolving deltas: 100% (11301/11301), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

# Replace 'path_to_your_zip_file' with the path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/Machine Learning Project/training.zip'

# Replace 'destination_folder' with the path where you want to unzip your files
destination_folder = '/content/Dataset'

# Create destination directory if it does not exist
os.makedirs(destination_folder, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dataset_dir = '/content/Dataset'
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')

Mounted at /content/drive


In [3]:
from skimage import exposure, io
import numpy as np
from skimage import img_as_ubyte
import shutil
from albumentations import Compose, HueSaturationValue, RandomBrightnessContrast
from skimage.color import rgba2rgb, gray2rgb


# Get a list of all image files
all_images = os.listdir(images_dir)

# Split the dataset into training and validation
train_images, val_images = train_test_split(all_images, test_size=0.2, random_state=42)


def augment_and_normalize_image(image):
    # Check if the image is grayscale or RGBA, and convert to RGB if necessary
    if image.ndim == 2:  # Grayscale
        image = gray2rgb(image)
    elif image.ndim == 3 and image.shape[2] == 4:  # RGBA to RGB
        image = rgba2rgb(image)

    # Ensure the image is uint8 before applying Albumentations augmentations
    image = img_as_ubyte(image)

    # Define augmentation pipeline
    transform = Compose([
        HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5)
    ])

    # Apply transformations
    augmented = transform(image=image)
    image = augmented['image']

    return image

def process_and_move_files(files, source_folder, dest_folder, augment=False):
    for file in files:
        # Process image
        image_path = os.path.join(source_folder, file)
        image = io.imread(image_path)

        # Apply augmentations if specified
        if augment:
            image = augment_and_normalize_image(image)

        # Save the processed image
        io.imsave(os.path.join(dest_folder, file), image)  # Save image to destination folder

        # Move corresponding label file
        label_file = file.replace('jpg', 'txt').replace('png', 'txt')
        shutil.move(os.path.join(source_folder.replace('images', 'labels'), label_file), dest_folder.replace('images', 'labels'))



# Process and move the files
train_dir = os.path.join(dataset_dir, 'images/train')
val_dir = os.path.join(dataset_dir, 'images/val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(train_dir.replace('images', 'labels'), exist_ok=True)
os.makedirs(val_dir.replace('images', 'labels'), exist_ok=True)

process_and_move_files(train_images, images_dir, train_dir, augment=True)  # Apply augmentation to training images
process_and_move_files(val_images, images_dir, val_dir)


This is the only parameter modification test here, changing the original 640 image size to a larger 1280 size. Test whether a larger image resolution can give the model better performance.

In [4]:
# Train YOLOv5 on custom dataset for a certain number of epochs
!python train.py --img 1280 --batch 16 --epochs 50 --data /content/Dataset/dataset.yaml --weights yolov5s.pt


2024-03-07 14:57:41.317078: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 14:57:41.317124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 14:57:41.318500: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/Dataset/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, si

# Model Performance Comparison Report: 640 vs 1280 Image Sizes

## Introduction

This report provides a comparative analysis between two different image sizes, 640 and 1280, used in training a deep learning object detection model. The objective is to assess the impact of image size on model performance metrics including Precision, Recall, mAP50, and mAP50-95.

## Overall Performance Comparison

| Metric     | Image Size 640 | Image Size 1280 |
|------------|----------------|-----------------|
| Precision  | 0.882          | 0.87            |
| Recall     | 0.839          | 0.791           |
| mAP50      | 0.901          | 0.852           |
| mAP50-95   | 0.643          | 0.614           |

## Performance by Class

### Room

| Metric     | Image Size 640 | Image Size 1280 |
|------------|----------------|-----------------|
| Precision  | 0.909          | 0.909           |
| Recall     | 0.892          | 0.85            |
| mAP50      | 0.941          | 0.896           |
| mAP50-95   | 0.679          | 0.654           |

### Window

| Metric     | Image Size 640 | Image Size 1280 |
|------------|----------------|-----------------|
| Precision  | 0.88           | 0.858           |
| Recall     | 0.803          | 0.759           |
| mAP50      | 0.881          | 0.831           |
| mAP50-95   | 0.53           | 0.511           |

### Door

| Metric     | Image Size 640 | Image Size 1280 |
|------------|----------------|-----------------|
| Precision  | 0.858          | 0.842           |
| Recall     | 0.821          | 0.763           |
| mAP50      | 0.881          | 0.828           |
| mAP50-95   | 0.72           | 0.677           |

## Analysis

Upon comparing the two image sizes, it is evident that the model trained with 640 image size generally outperforms the one trained with 1280 image size across all main performance metrics. Specifically:

- The **Precision** sees a slight drop from 640 to 1280, indicating a marginal decrease in the proportion of true positive detections.
- The **Recall** metric shows a more significant decrease, suggesting that the model with 1280 image size is less capable of identifying all relevant instances in the dataset.
- **mAP50** and **mAP50-95** both decrease as the image size increases, which implies that the model's ability to accurately detect and localize objects diminishes with larger image sizes.

### Considerations

- **Computational Load**: The increased image size leads to higher computational requirements and longer inference times, which might not be justifiable given the decrease in performance metrics.
- **Data Representation**: Larger image sizes could introduce more complexity and variability that the current model architecture or training regimen may not handle optimally.
- **Optimization and Tuning**: The model might require different tuning or a different architecture to fully leverage the higher resolution provided by the 1280 image size.

## Conclusion

The comparative analysis between the 640 and 1280 image sizes demonstrates that, for this specific object detection model, a smaller image size of 640 provides better performance across several key metrics. While larger image sizes can theoretically offer more detailed information for object detection, they also pose greater challenges for model training and may require more computational resources. Future work should focus on optimizing model parameters and architectures to better accommodate larger image sizes or consider the trade-offs between resolution and performance for their specific application requirements.
